# 병변 검출 AI 경진대회 (Object Detection)

## Overview

Object Detection 대회를 제대로 참여한 건 처음이었는데 운 좋게 수상할 수 있었던 것 같습니다. 특히 Public에서는 6등이었는데 Private에서 3등이 될 수 있었던 가장 큰 요인은 YoloV5와 MMDetection의 모델들을 사용하여 앙상블 시 다양성을 준 것이 주요한 요인이지 않았나 싶습니다.

- 학습에 사용한 모델은 아래와 같습니다.
    - YoloV5 - Ensemble 시 다양성을 주기위해 이미지 사이즈, 모델 사이즈, Fold를 다양하게 학습하였습니다.
    - 다만, 적절한 구조 및 하이퍼파라미터를 찾는것에 시간을 소모하여 Fold0인 모델이 많습니다.
        - L - Fold0: size(384, 512)
        - X - Fold0: size(320, 384, 480), Fold2(480), Fold4(640)

    - MMDetection
        - CenterNet(R-18) - 512size (Fold0, Fold1)
        - RetinaNet(R-101) - 1024size (Fold0)

    <br>

    - 모든 모델은 학습 시 Pretrained Weight을 yolov5 & mmdet github로부터 받아서 사용하였습니다.

- 추론
    - 모델
        - 7개의 YoloV5 모델 Single image inference = 7개의 결과
        - 7개의 YoloV5 모델 Test Time Augmentation(TTA) = 7개의 결과
        - 2개의 CenterNet Single image inference = 2개의 결과
        - 2개의 CenterNet Test Time Augmentation(TTA) = 2개의 결과
        - 1개의 RetinaNet Single image inference = 1개의 결과 (TTA는 꽤 오래걸려서 수행하지 않았습니다)
    
    - Ensemble
        - BBox를 Ensemble하는 방법으로는 캐글에서도 많이 사용되는 Weighted Boxes Fusion (https://github.com/ZFTurbo/Weighted-Boxes-Fusion)라는 기법을 사용하였습니다.


<br>

- 최종적으로 총 10개의 모델을 학습하고 TTA를 포함하여 19개의 결과물을 Ensemble하였습니다.
- 제 Github에서 trained Weight을 다운로드하여 결과를 추론해보실 수 있습니다.

## 0. Prerequisites

## 0-1. Requirements
- Ubuntu 18.04, Cuda 11.1
- Anaconda - Python 3.8

- numpy
- pandas
- opencv-python
- python-dateutil
- pytz
- six
- matplotlib
- natsort
- tqdm
- scikit-learn
- ensemble-boxes
- torch==1.9.0 torchvision 0.10.0 with cuda 11.1 (YoloV5)
- torch==1.8.0 torchvision 0.9.0 with cuda 11.1 (MMDetection)
- mmcv-full
- mmdet

YoloV5의 경우 학습 속도가 torch 1.9.0 버전에서 더 빨라서 1.9.0을 사용하였고, MMDetection의 경우 1.9.0에서 코드가 돌아가지 않는 현상이 발생하여 1.8.0으로 다르게 사용하였습니다.

make_dataset.sh나 install_mmdet.sh에서 필요한 라이브러리가 설치되도록 하였습니다.

## 0-2 Directory 구조
- data/ 폴더에 train, test폴더의 json파일과 lesion.names, 그리고 etc/ 폴더의 lesion.yaml파일을 준비하면 됩니다.

```bash
.
├── README.md
├── data
│   ├── class_id_info.csv
│   ├── lesion.names
│   ├── sample_submission.csv
│   └── train
        ├── train_100000.json
        ├── train_100001.json
│   ├── test
        ├── test_200000.json
        ├── test_200001.json

├── convert2Yolo
│   └── requirements.txt
│   ├── ...

├── etc
│   ├── lesion_0f.yaml
│   ├── lesion_2f.yaml
│   └── lesion_4f.yaml

└── yolov5
    ├── data
    ├── train.py
    ├── detect.py
    ...

├── mmdet_configs
│   ├── coco_detection.py
│   ├── coco_detection_1f.py
│   ├── ...

├── mmdetection
│   ├── mmdet
│   ├── tools
│   └── work_dirs
│   ├── ...

├── pretrain_weights
│   ├── centernet_resnet18_dcnv2_140e_coco_20210702_155131-c8cd631f.pth
│   └── retinanet_r101_fpn_2x_coco_20200131-5560aee8.pth

├── trained_weights
│   ├── exp1.pt
│   ├── exp2.pt
│   ├── ...

```

#### 아래 코드들은 모두 Ubuntu terminal에서 실행하여야 합니다.
#### 자세한 코드는 제 Github에서 확인해주시면 감사하겠습니다. (https://github.com/wooseok-shin/Lesion-Detection-3rd-place-solution)

## 1. Make dataset

- 1. 필요한 라이브러리와 본 대회의 데이터 Fromat에서 COCO 데이터 Format으로 변형하여 줍니다. (이여름님의 코드 공유를 참고하였습니다. 감사합니다! https://dacon.io/competitions/official/235855/codeshare/3729?page=1&dtype=recent)

- 2. 이후 COCO Format은 MMdetection에 사용하고, YoloV5 Format을 맞춰주기 위해 https://github.com/ssaru/convert2Yolo에서 배포하는 코드를 Clone하여 사용하였습니다.

- 3. YoloV5 Format 데이터를 train/valid로 split해줍니다.

- 4. YoloV5의 Repository를 clone하고 YAML 파일을 만들어줍니다. (제 GitHub etc폴더에 올려져 있습니다.)

In [ ]:
# sh make_dataset.sh
pip install Pillow cycler kiwisolver numpy pandas opencv-python python-dateutil pytz six matplotlib natsort tqdm scikit-learn ensemble-boxes
pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html

# 1. Make coco format (for mmdetection & yolov5)
python convert_coco_format.py

# 2. Coco to Yolov5 format dataset
git clone https://github.com/ssaru/convert2Yolo.git
cd convert2Yolo
mkdir ../data/yolo_labels/
## lesion.names 파일을 ../data/ 위치에 만들어주어야 함. (현재는 만들어져있음.)
python example.py --datasets COCO --img_path ../data/train_imgs/ --label ../data/annos/train_annotations_full.json --convert_output_path ../data/yolo_labels/ --img_type ".png" --manifest_path ./ --cls_list_file ../data/lesion.names
cd ..

# 3. Yolov5 train/valid split
python split_yolo_dataset.py

# 4. Yolov5 clone & Prerequisite
git clone https://github.com/ultralytics/yolov5.git
cd yolov5
pip install -r requirements.txt
cp ../etc/* ./data/    # Yolov5 학습을 위해 yolov5/data/경로에 yaml 파일을 만들어 주어야함 (학습에 필요한 경로 설정) - 미리 첨부해놓은 파일 복사해서 해당 폴더에 넣어주기


## 2. Train

### 1) Train YoloV5 models (Yolov5 L,X)

In [ ]:
# sh train_yolov5.sh
# Train seven models

## Ensemble 시 다양성을 많이 주기위해 이미지 사이즈, 모델 사이즈, Fold를 다양하게 학습
cd yolov5

### 2 x yolov5l (0Fold: 384,512 size)
python train.py --project weights/ --name=exp1 --img 384 --batch 16 --epochs 125 --data lesion_0f.yaml --weights yolov5l.pt --save-period 5 --workers 4
python train.py --project weights/ --name=exp2 --img 512 --batch 16 --epochs 200 --data lesion_0f.yaml --weights yolov5l.pt --save-period 5 --workers 4

### 5 x yolov5x (0Fold: 320,384,480 size & 2,4Fold)
python train.py --project weights/ --name=exp3 --img 320 --batch 16 --epochs 125 --data lesion_0f.yaml --weights yolov5x.pt --save-period 5 --workers 4
python train.py --project weights/ --name=exp4 --img 384 --batch 16 --epochs 125 --data lesion_0f.yaml --weights yolov5x.pt --save-period 5 --workers 4
python train.py --project weights/ --name=exp5 --img 480 --batch 16 --epochs 125 --data lesion_0f.yaml --weights yolov5x.pt --save-period 5 --workers 4
python train.py --project weights/ --name=exp6 --img 480 --batch 16 --epochs 125 --data lesion_2f.yaml --weights yolov5x.pt --save-period 5 --workers 4
python train.py --project weights/ --name=exp7 --img 640 --batch 16 --epochs 125 --data lesion_4f.yaml --weights yolov5x.pt --save-period 5 --workers 4

### 2) Train MMDetection models (CenterNet, RetinaNet)
- 기존에 Yolov5를 학습시킬 때 torch 1.9.0 버전을 사용하였는데, MMDetection에서 1.9.0이 돌아가지 않는 현상이 있었습니다.
- 따라서, torch를 1.8.0, torchvision을 0.10.0으로 재설치 해주었습니다. (가상환경을 따로할 시 상관없음)


In [ ]:
# sh mmdet_install.sh (mmcv 및 mmdet 설치)

# Install mmcv and mmdet
pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 -f https://download.pytorch.org/whl/torch_stable.html
pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.8.0/index.html
pip install mmdet

# MMDetection을 처음부터 다운로드 받아서 할 시 Git Clone 수행
git clone https://github.com/open-mmlab/mmdetection.git

- 처음엔 Yolov5 계열로만 Ensemble을 하다가 성능 향상이 더뎌진 것 같아 전혀 다른 구조의 모델을 사용하였습니다.
- 그 중에서 CenterNet(R-18)과 RetinaNet(R-101)을 학습하였습니다.
- Backbone으로 CBNetV2나 Detector로 HTC 등 COCO 기준으로 더 좋은 성능을 보이는 모델도 학습해보았으나 오버피팅 문제가 발생하여 가벼운 구조를 채택하였습니다.

In [ ]:
# sh train_mmdet.sh
# Train three models
cp -r mmdet_configs/ mmdetection/configs/    # 수정해놓은 config 폴더 복사해서 mmdetection/configs 폴더에 넣기

# Download pretrain weight from mmdetection github (CenterNet R-18 and Retina R-101)
wget https://download.openmmlab.com/mmdetection/v2.0/centernet/centernet_resnet18_dcnv2_140e_coco/centernet_resnet18_dcnv2_140e_coco_20210702_155131-c8cd631f.pth -P ./pretrain_weights/
wget https://download.openmmlab.com/mmdetection/v2.0/retinanet/retinanet_r101_fpn_2x_coco/retinanet_r101_fpn_2x_coco_20200131-5560aee8.pth -P ./pretrain_weights/

cd mmdetection

## 1.Centernet Fold 0
bash tools/dist_train.sh configs/mmdet_configs/exp8_centernet_resnet18_dcnv2_140e_coco.py 1

## 2.Centernet Fold 1
bash tools/dist_train.sh configs/mmdet_configs/exp9_centernet_resnet18_dcnv2_140e_coco.py 1

## 3.Retina ResNet-101 Fold 0
bash tools/dist_train.sh configs/mmdet_configs/exp10_retinanet_r101_fpn_1x_coco.py 1


## 3. Inference

### 1) YoloV5 models (7 single inference + 7 TTA)

In [ ]:
# sh inference_yolo.sh
cd yolov5

# yolov5/results/expN 폴더에 txt파일 예측 결과물 저장

## Single image inference 7 models
python detect.py --weights weights/exp1/weights/epoch120.pt --project ../results/ --name exp1_single --img 384 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave
python detect.py --weights weights/exp2/weights/epoch150.pt --project ../results/ --name exp2_single --img 512 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave
python detect.py --weights weights/exp3/weights/epoch120.pt --project ../results/ --name exp3_single --img 320 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave
python detect.py --weights weights/exp4/weights/epoch120.pt --project ../results/ --name exp4_single --img 384 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave
python detect.py --weights weights/exp5/weights/epoch120.pt --project ../results/ --name exp5_single --img 480 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave
python detect.py --weights weights/exp6/weights/epoch120.pt --project ../results/ --name exp6_single --img 480 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave
python detect.py --weights weights/exp7/weights/epoch120.pt --project ../results/ --name exp7_single --img 640 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave

## TTA 7 models
python detect.py --weights weights/exp1/weights/epoch120.pt --project ../results/ --name exp1_tta --img 384 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave --augment
python detect.py --weights weights/exp2/weights/epoch150.pt --project ../results/ --name exp2_tta --img 512 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave --augment
python detect.py --weights weights/exp3/weights/epoch120.pt --project ../results/ --name exp3_tta --img 320 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave --augment
python detect.py --weights weights/exp4/weights/epoch120.pt --project ../results/ --name exp4_tta --img 384 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave --augment
python detect.py --weights weights/exp5/weights/epoch120.pt --project ../results/ --name exp5_tta --img 480 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave --augment
python detect.py --weights weights/exp6/weights/epoch120.pt --project ../results/ --name exp6_tta --img 480 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave --augment
python detect.py --weights weights/exp7/weights/epoch120.pt --project ../results/ --name exp7_tta --img 640 --source ../data/test_imgs --save-txt --save-conf --conf-thres 0.001 --iou-thres 0.6 --nosave --augment

cd ../

# Yolov5 infernece format to submission format
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp1_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp2_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp3_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp4_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp5_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp6_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp7_single --test_img_path data/test_imgs/

python inference_yolo_agg.py --dir_prefix results/ --exp_name exp1_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp2_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp3_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp4_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp5_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp6_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp7_tta --test_img_path data/test_imgs/


### 2) MMDetection models (3 single + 2 TTA)

In [ ]:
# sh inference_mmdet.sh

# Single image inference 3 models (각각 25epoch, 28epoch, 22epoch에서 Best valid score가 나옴)
python inference_mmdet.py --exp_name exp8_centernet_resnet18_dcnv2_140e_coco --checkpoint epoch_25
python inference_mmdet.py --exp_name exp9_centernet_resnet18_dcnv2_140e_coco --checkpoint epoch_28
python inference_mmdet.py --exp_name exp10_retinanet_r101_fpn_1x_coco --checkpoint epoch_22

# TTA 2 models
python inference_mmdet.py --use_tta _tta --exp_name exp8_centernet_resnet18_dcnv2_140e_coco --checkpoint epoch_25
python inference_mmdet.py --use_tta _tta --exp_name exp9_centernet_resnet18_dcnv2_140e_coco --checkpoint epoch_28



## 4. WBF Ensemble

## WBF Ensemble - 7 Yolo models x 2(NoTTA, TTA) + 2 CenterNet x 2(NoTTA, TTA) + 1 RetinaNet
- Make final submission

In [ ]:
python ensemble_wbf.py

## * Public, Private Score 복원: 학습된 모델 Weight를 불러와서 inference하기 (Git으로부터 Weights Download)
- 10개의 Weight를 불러와야하는데 Github 서버 문제인지 가끔 하나씩 안불러오는 경우가 있는 것 같습니다.
- exp1~exp10까지 다 불러와졌는지 확인이 필요합니다.
- 한 두개가 빠져있으면 해당 폴더를 삭제하고 다시 실행하거나 하나씩 wget하면 됩니다.

In [ ]:
# Download trained weight from my github (https://github.com/wooseok-shin/Lesion-Detection-3rd-place-solution)
wget -i https://raw.githubusercontent.com/wooseok-shin/Lesion-Detection-3rd-place-solution/main/load_trained_weight.txt -P trained_weights


# Yolov5 7x2 models
cd yolov5
# yolov5/results/expN 폴더에 txt파일 예측 결과물 저장

## Single image inference 7 models
python detect.py --weights ../trained_weights/exp1.pt --project ../results/ --name exp1_single --img 384 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6
python detect.py --weights ../trained_weights/exp2.pt --project ../results/ --name exp2_single --img 512 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6
python detect.py --weights ../trained_weights/exp3.pt --project ../results/ --name exp3_single --img 320 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6
python detect.py --weights ../trained_weights/exp4.pt --project ../results/ --name exp4_single --img 384 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6
python detect.py --weights ../trained_weights/exp5.pt --project ../results/ --name exp5_single --img 480 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6
python detect.py --weights ../trained_weights/exp6.pt --project ../results/ --name exp6_single --img 480 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6
python detect.py --weights ../trained_weights/exp7.pt --project ../results/ --name exp7_single --img 640 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6

## TTA 7 models
python detect.py --weights ../trained_weights/exp1.pt --project ../results/ --name exp1_tta --img 384 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6 --augment
python detect.py --weights ../trained_weights/exp2.pt --project ../results/ --name exp2_tta --img 512 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6 --augment
python detect.py --weights ../trained_weights/exp3.pt --project ../results/ --name exp3_tta --img 320 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6 --augment
python detect.py --weights ../trained_weights/exp4.pt --project ../results/ --name exp4_tta --img 384 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6 --augment
python detect.py --weights ../trained_weights/exp5.pt --project ../results/ --name exp5_tta --img 480 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6 --augment
python detect.py --weights ../trained_weights/exp6.pt --project ../results/ --name exp6_tta --img 480 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6 --augment
python detect.py --weights ../trained_weights/exp7.pt --project ../results/ --name exp7_tta --img 640 --source ../data/test_imgs --save-txt --save-conf --nosave --conf-thres 0.001 --iou-thres 0.6 --augment

cd ../

## Yolov5 infernece format to submission format
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp1_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp2_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp3_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp4_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp5_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp6_single --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp7_single --test_img_path data/test_imgs/

python inference_yolo_agg.py --dir_prefix results/ --exp_name exp1_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp2_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp3_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp4_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp5_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp6_tta --test_img_path data/test_imgs/
python inference_yolo_agg.py --dir_prefix results/ --exp_name exp7_tta --test_img_path data/test_imgs/


# MMdetection 5 models
cp -r mmdet_configs/ mmdetection/configs/ # Configs 복사 및 덮어쓰기

## Single image inference 3 models
python inference_load_mmdet.py --exp_name exp8_centernet_resnet18_dcnv2_140e_coco --checkpoint exp8
python inference_load_mmdet.py --exp_name exp9_centernet_resnet18_dcnv2_140e_coco --checkpoint exp9
python inference_load_mmdet.py --exp_name exp10_retinanet_r101_fpn_1x_coco --checkpoint exp10

## TTA 2 models
python inference_load_mmdet.py --use_tta _tta --exp_name exp8_centernet_resnet18_dcnv2_140e_coco --checkpoint exp8
python inference_load_mmdet.py --use_tta _tta --exp_name exp9_centernet_resnet18_dcnv2_140e_coco --checkpoint exp9

# Final WBF Ensemble
python ensemble_wbf.py